# Code for AISTATS submission (Reconstructing Test Labels from Noisy Loss Functions)
## multi-query label inference with log-loss and softmax-loss on MNIST

### Experiment setup

In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
from collections import Counter
import math
import time
import warnings
warnings.filterwarnings('ignore')

np.random.seed(12345)

In [2]:
all_labels = dict() # a dictionary to hold all labels from real datasets
os.listdir('data')

['.DS_Store', 'iris.csv', 'mnist.csv', 'titanic.csv', 'satimage.csv']

### MNIST

In [3]:
mnist = pd.read_csv('data/mnist.csv')
mnist = mnist.sample(frac=1)
print(mnist.shape) 
print(Counter(mnist['class']))
mnist.head()

(70000, 785)
Counter({1: 7877, 7: 7293, 3: 7141, 2: 6990, 9: 6958, 0: 6903, 6: 6876, 8: 6825, 4: 6824, 5: 6313})


pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
58332       0       0       0       0       0       0       0       0       0   
53059       0       0       0       0       0       0       0       0       0   
36369       0       0       0       0       0       0       0       0       0   
9078        0       0       0       0       0       0       0       0       0   
4675        0       0       0       0       0       0       0       0       0   

       pixel10  ...  pixel776  pixel777  pixel778  pixel779  pixel780  \
58332        0  ...         0         0         0         0         0   
53059        0  ...         0         0         0         0         0   
36369        0  ...         0         0         0         0         0   
9078         0  ...         0         0         0         0         0   
4675         0  ...         0         0         0         0         0   

       pixel781  pixel782  pixel783  pixel784  class  
58332         0         0         0         0      9  
53059         0         0         0         0      9  
36369         0         0         0         0      9  
9078          0         0         0         0      3  
4675          0         0         0         0      8  

[5 rows x 785 columns]

In [4]:
mnist_label = mnist['class'].apply(lambda x: int(x)).tolist()
Counter(mnist_label)

Counter({9: 6958,
         3: 7141,
         8: 6825,
         0: 6903,
         2: 6990,
         6: 6876,
         5: 6313,
         4: 6824,
         7: 7293,
         1: 7877})

## utility functions in the experiments

In [5]:
# function to compute the k-ary-logloss score
def k_ary_logloss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the k-ary-logloss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        loss -= math.log(cur_pred)
    return loss / N

In [6]:
# function to compute the sigmoid loss
def sigmoid_loss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the sigmoid loss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        sig = 1 / (1 + np.exp(-cur_pred))
        loss -= math.log(sig)
    return loss / N

In [7]:
# function to compute the sigmoid loss
def softmax_loss(labeling=[], prediction=np.array([[]])):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    Returns the sigmoid loss score
    """
    N = len(labeling)
    loss = 0
    for i in range(N):
        cur_pred = prediction[i, labeling[i]]
        cur_num = np.exp(cur_pred)
        cur_den = np.sum(np.exp(prediction[i, :]))
        cur_softmax = cur_num / cur_den
        loss -= math.log(cur_softmax)
    return loss / N

In [8]:
# wrapper loss function
def loss(labeling=[], prediction=np.array([[]]), tau=0, loss_type='logloss'):
    """
    labeling: 1-d list with class labels represented by integer (starts from 0)
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    tau: noise
    loss_type: logloss, sigmoidloss, or softmaxloss
    Returns the loss score
    """
    if loss_type == 'logloss':
        return k_ary_logloss(labeling, prediction) + tau * np.random.rand()
    if loss_type == 'sigmoidloss':
        return sigmoid_loss(labeling, prediction) + tau * np.random.rand()
    if loss_type == 'softmaxloss':
        return softmax_loss(labeling, prediction) + tau * np.random.rand()

In [9]:
# function to generate the predictions
def generate_prediction(N, K, tau_max):
    """
    N: number of data points
    K: number of classes
    tau_max: maximum noise
    Return a 2-d numpy array of predictions
    """
    prediction = np.zeros([N, K])
    for i in range(N):
        n = i + 1
        for j in range(K):
            k = j + 1
            prediction[i, j] = 2 ** ((n-1) * K + k) * N * tau_max
        
        diff = prediction[i, :] - max(prediction[i, :])
        prediction[i, :] = 3 ** diff
        prediction[i, :] /= sum(prediction[i, :])
        for j in range(K):
            prediction[i, j] = sys.float_info.min if prediction[i, j] < sys.float_info.min \
                else prediction[i, j]
    return prediction 

In [10]:
# generator for K-nary labelings
def gen_K_ary(nbits, K):
    """
    nbits: number of bits in the K-nary vector
    K: K in K-nary, e.g. K=2 is binary, K=3 is ternary
    Returns a generator for all K-nary labelings of length nbits
    """
    for i in range(K**nbits):
        bin_str = str(np.base_repr(i, K))
        l = len(bin_str)
        bin_arr = np.zeros(nbits)
        for j in range(l):
            bin_arr[nbits-j-1] = float(bin_str[l-j-1])
        yield [int(x) for x in bin_arr]

In [11]:
# the main label inference algorithm
def main_algorithm(noisy_loss, N, K, prediction, loss_type):
    """
    noisy_loss: observed loss score with noise
    N: number of data points
    K: number of classes
    prediction: 2-d numpy array, where prediction[i,k] represents the prediction for data i in class k
    loss_type: either logloss or sigmoidloss
    Returns the inferred labeling
    """
    label_infer = []
    diff_min = np.inf
    lab_generator = gen_K_ary(N, K)
    
    while True:
        try:
            label_cur = next(lab_generator) 
            loss_cur = loss(label_cur, prediction, 0, loss_type)
            diff_cur = np.abs(loss_cur - noisy_loss) 
            if diff_cur < diff_min:
                label_infer = label_cur.copy()
                diff_min = diff_cur
        except:
            break
    return label_infer

In [12]:
# the main function for simulation
def main_sim(true_labeling, K, tau_max, loss_type, shard_size, max_N_feasible):
    """
    true_labeling: ground truth labeling
    K: number of classes
    tau_max: maximum noise
    loss_type: either logloss or sigmoidloss
    shard_size: number of data points in each shard
    max_N_feasible: largest N such that the label inference algorithm is feasible
    Returns the average accuracy and the number of shards
    """
    acc = 0
    mean_loss = 0
    n_total = len(true_labeling)  
    n_shard = int(np.ceil(n_total / shard_size))
    for j in range(n_shard):
        if j < n_shard - 1:
            cur_shard_size = shard_size
            labeling = true_labeling[j*shard_size : (j+1)*shard_size]
        else:
            cur_shard_size = n_total - j*shard_size
            labeling = true_labeling[j*shard_size : n_total]
        N = int(min(max_N_feasible, cur_shard_size))
        labeling_cur = labeling[:N]
        tau = 2 * np.exp(1) * tau_max / (np.exp(1) - 1)
        prediction = generate_prediction(N, K, tau)
        noisy_loss = loss(labeling_cur, prediction, tau_max, loss_type)
        labeling_infer = main_algorithm(noisy_loss, N, K, prediction, loss_type)
        acc += sum([x == y for x, y in zip(labeling_infer, labeling_cur)])
    return acc / n_total, n_shard

## label inference experiments

In [13]:
# logloss experiment
loss_type = 'logloss'
result_logloss = []

# read in the max_N_feasible for each K and tau_max setting from previous experimental results
setting_logloss = pd.read_csv('logloss_singlequery_mnist.csv')
 
df = setting_logloss.dropna().groupby(['K','tau_max'])['N'].max()
df = df.to_frame()
df = df.reset_index()
df.head()

K  tau_max  N
0  10   0.0001  3
1  10   0.0100  2
2  10   0.1000  2
3  10   1.0000  1

In [14]:
prev_acc = 0
for _, rows in df.iterrows():
    K, tau_max, max_N_feasible = int(rows[0]), rows[1], int(rows[2])
    shard_size_vec = [1,2,7,10,70,100,700,1000,7000,10000,35000,70000]
    shard_size_vec = list(reversed(shard_size_vec))
    
    n_total = len(mnist_label)
    for i, shard_size in enumerate(shard_size_vec):
        start = time.time()
        acc, n_shards = main_sim(mnist_label, K, tau_max, loss_type, shard_size, max_N_feasible)
        if i > 0:
            acc = max(acc, prev_acc)
        seconds = time.time() - start
        result_logloss.append((K, tau_max, acc, n_shards, seconds))
        print(K, tau_max, acc, n_shards, seconds)
        prev_acc = acc

10 0.0001 4.2857142857142856e-05 1 0.014373302459716797
10 0.0001 5.714285714285714e-05 2 0.027696847915649414
10 0.0001 0.00017142857142857143 7 0.09381985664367676
10 0.0001 0.00025714285714285715 10 0.1400771141052246
10 0.0001 0.001542857142857143 70 0.7966721057891846
10 0.0001 0.001957142857142857 100 1.0333759784698486
10 0.0001 0.0138 700 7.530750036239624
10 0.0001 0.019671428571428572 1000 10.347013235092163
10 0.0001 0.1395142857142857 7000 74.19913125038147
10 0.0001 0.1987 10000 103.43735671043396
10 0.0001 0.8428714285714286 35000 34.8848819732666
10 0.0001 1.0 70000 8.989228963851929
10 0.01 2.857142857142857e-05 1 0.0011858940124511719
10 0.01 5.714285714285714e-05 2 0.0023589134216308594
10 0.01 0.00012857142857142858 7 0.007035255432128906
10 0.01 0.00021428571428571427 10 0.009757041931152344
10 0.01 0.0011857142857142858 70 0.06871199607849121
10 0.01 0.0017142857142857142 100 0.09604787826538086
10 0.01 0.011928571428571429 700 0.6788349151611328
10 0.01 0.0171 100

In [15]:
df_logloss = pd.DataFrame(result_logloss)
df_logloss.columns = ['K', 'tau_max', 'acc', 'n_shards', 'seconds']
df_logloss.to_csv('logloss_multiquery_mnist.csv', index=False)
print(df_logloss.shape)
df_logloss.head(10)

(48, 5)


K  tau_max       acc  n_shards     seconds
0  10   0.0001  0.000043         1    0.014373
1  10   0.0001  0.000057         2    0.027697
2  10   0.0001  0.000171         7    0.093820
3  10   0.0001  0.000257        10    0.140077
4  10   0.0001  0.001543        70    0.796672
5  10   0.0001  0.001957       100    1.033376
6  10   0.0001  0.013800       700    7.530750
7  10   0.0001  0.019671      1000   10.347013
8  10   0.0001  0.139514      7000   74.199131
9  10   0.0001  0.198700     10000  103.437357

In [16]:
# softmaxloss experiment
loss_type = 'softmaxloss'
result_softmaxloss = []

# read in the max_N_feasible for each K and tau_max setting from previous experimental results
setting_softmaxloss = pd.read_csv('softmaxloss_singlequery_mnist.csv')
 
df = setting_softmaxloss.dropna().groupby(['K','tau_max'])['N'].max()
df = df.to_frame()
df = df.reset_index()

In [17]:
for _, rows in df.iterrows():
    K, tau_max, max_N_feasible = int(rows[0]), rows[1], int(rows[2])
    shard_size_vec = [1,2,7,10,70,100,700,1000,7000,10000,35000,70000]
    shard_size_vec = list(reversed(shard_size_vec))
    
    n_total = len(mnist_label)
    for i, shard_size in enumerate(shard_size_vec):
        start = time.time()
        acc, n_shards = main_sim(mnist_label, K, tau_max, loss_type, shard_size, max_N_feasible)
        if i > 0:
            acc = max(acc, prev_acc)
        seconds = time.time() - start
        result_softmaxloss.append((K, tau_max, acc, n_shards, seconds))
        print(K, tau_max, acc, n_shards, seconds)
        prev_acc = acc

10 0.0001 2.857142857142857e-05 1 0.002897024154663086
10 0.0001 4.2857142857142856e-05 2 0.005511045455932617
10 0.0001 0.0001 7 0.01937699317932129
10 0.0001 0.00018571428571428572 10 0.027550220489501953
10 0.0001 0.0009714285714285714 70 0.1922290325164795
10 0.0001 0.0016 100 0.28182101249694824
10 0.0001 0.010614285714285715 700 1.9026658535003662
10 0.0001 0.015471428571428572 1000 2.663635015487671
10 0.0001 0.10718571428571429 7000 19.431358098983765
10 0.0001 0.1541142857142857 10000 27.19309902191162
10 0.0001 0.5387285714285714 35000 99.89856886863708
10 0.0001 0.8890285714285714 70000 17.442226886749268
10 0.01 1.4285714285714285e-05 1 0.0005121231079101562
10 0.01 1.4285714285714285e-05 2 0.0008459091186523438
10 0.01 2.857142857142857e-05 7 0.006310224533081055
10 0.01 4.2857142857142856e-05 10 0.0029838085174560547
10 0.01 0.00015714285714285713 70 0.019643068313598633
10 0.01 0.00028571428571428574 100 0.027709007263183594
10 0.01 0.0019285714285714286 700 0.1683750152

In [18]:
df_softmaxloss = pd.DataFrame(result_softmaxloss)
df_softmaxloss.columns = ['K', 'tau_max', 'acc', 'n_shards', 'seconds']
df_softmaxloss.to_csv('softmaxloss_multiquery_mnist.csv', index=False)
print(df_softmaxloss.shape)
df_softmaxloss.head(10)

(48, 5)


K  tau_max       acc  n_shards    seconds
0  10   0.0001  0.000029         1   0.002897
1  10   0.0001  0.000043         2   0.005511
2  10   0.0001  0.000100         7   0.019377
3  10   0.0001  0.000186        10   0.027550
4  10   0.0001  0.000971        70   0.192229
5  10   0.0001  0.001600       100   0.281821
6  10   0.0001  0.010614       700   1.902666
7  10   0.0001  0.015471      1000   2.663635
8  10   0.0001  0.107186      7000  19.431358
9  10   0.0001  0.154114     10000  27.193099